In [12]:
import arcpy
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *
from arcpy import env
import os
import pandas as pd
import numpy as np

In [2]:
# 读取全数据点
point_data = r"D:\ArcgisData\pred_organic_p_n\pred_database\Oragnic_Basedata.gdb\GD_POINT_NPSOM"

In [8]:
# 特征数据
fetatus_data = r"D:\ArcgisData\basedata\basetrain_5m.gdb"

In [9]:
# 设置工作环境
env.workspace = fetatus_data

In [10]:
# 采样
def sample_point(point_,raster_,out_name):
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    TableToTable(table_,out_path,out_name)
    return None
    

In [11]:
# 获取栅格列表
raster_list = arcpy.ListRasters()

In [ ]:
table_gdb_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\tableresult.gdb"
# 使用Delete_management函数删除数据库中的所有内容
arcpy.Delete_management(table_gdb_path)
# 创建一个数据库
arcpy.management.CreateFileGDB(r"D:\ArcgisData\pred_organic_p_n\feature_table", "tableresult", "CURRENT")

In [ ]:
# 逐个采样
for one_raster in raster_list:
    sample_point(point_data,one_raster,os.path.join(table_gdb_path,one_raster))

In [19]:
env.workspace = table_gdb_path

In [50]:
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,["OBJECTID","Ph","N","P","有机质含量"]))
result_df.rename(columns={"OBJECTID":"GD_POINT_NPSOM"},inplace=True)
tables = arcpy.ListTables()
# 存储每个表的最后一个字段的数据
merged_data = []

# 读取每个表的最后一个字段的数据
for table in tables:
    
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))
    # 提取最后一个字段的数据
    merged_df = df[['GD_POINT_NPSOM', df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=['GD_POINT_NPSOM'])

In [52]:

result_df.columns

Index(['GD_POINT_NPSOM', 'Ph', 'N', 'P', '有机质含量', 'TWI_5_Band_1',
       'TPI_201_Band_1', 'TPI_101_Band_1', 'TPI_11_Band_1', 'TPI_3_Band_1',
       'TMP_Band_1', 'SOILQS_Band_1', 'SLOP_Band_1', 'PRE_Band_1',
       'NIGTH_Band_1', 'NDVI_Band_1', 'DEM_Band_1', 'CUR_Band_1', 'ASP_Band_1',
       'PLCUR_Band_1', 'POCUR_Band_1', 'OSJL_Band_1', 'LAT_Band_1',
       'LON_Band_1', 'DZ_Band_1', 'DL_Band_1'],
      dtype='object')

In [56]:
new_columns = [str(_).replace("_Band_1","").replace("_","").replace("有机质含量","SOM") for _ in result_df.columns]

In [58]:
result_df.rename(columns=dict(zip(result_df.columns, new_columns)))

,GDPOINTNPSOM,Ph,N,P,SOM,TWI5,TPI201,TPI101,TPI11,TPI3,...,DEM,CUR,ASP,PLCUR,POCUR,OSJL,LAT,LON,DZ,DL
0,1,5.32,1486.0,1252.0,24.1,5.962960,15.477661,10.274841,0.157227,0.025696,...,885.099976,4.000000,277.943970,-0.644262,-4.644262,66.708321,2738470.0,541870.0,2,3
1,2,5.85,1402.0,929.0,24.2,8.513925,20.920898,-20.338257,-4.790527,-0.400513,...,1270.500000,4.000000,71.036064,-3.452797,-7.452796,247.487381,3045130.0,417910.0,2,3
2,3,6.97,1885.0,1110.0,29.8,6.150497,-70.393555,-31.960815,-1.263062,-0.061768,...,1162.699951,-4.000977,63.435398,-1.600421,2.400555,5.000000,3046240.0,419590.0,2,3
3,4,8.24,1623.0,538.0,27.1,5.421333,-12.195618,-14.295593,0.508789,0.033875,...,1013.599976,2.799561,84.494598,-0.051160,-2.850720,14.142136,3040240.0,423340.0,2,3
4,5,6.72,1325.0,562.0,18.7,6.967052,57.751343,22.754883,-0.085693,0.006958,...,1084.000000,-0.800293,150.833389,0.417779,1.218072,155.724121,3038980.0,420880.0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6803,6804,5.53,1908.0,628.0,34.9,7.875397,-32.677429,-13.003418,-0.095154,-0.006409,...,940.099976,-0.000488,45.000000,0.099792,0.100281,106.066017,2711890.0,454120.0,1,7
6804,6805,5.74,1552.0,901.0,25.9,6.314113,-31.092773,-6.552490,-0.225037,-0.023010,...,965.000000,-0.000000,280.304626,-0.748855,-0.748855,70.000000,2711650.0,455350.0,1,3
6805,6806,6.21,2013.0,526.0,31.9,3.450914,-68.870117,-33.270508,-1.991089,-0.258423,...,1357.199951,-4.400391,325.855774,-1.741844,2.658546,95.131485,3151870.0,478180.0,6,6
6806,6807,7.43,3087.0,788.0,48.5,6.215333,-59.951660,-15.557617,0.005493,-0.001587,...,1100.800049,-0.399414,191.307785,-0.356667,0.042747,264.196899,2970250.0,723100.0,8,3


In [ ]:
# 存储位置
feature_out_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\feature_table_result"
result_df.to_csv(os.path.join(feature_out_path,"all_result.csv"),index=False)